[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/schlichtanders/fall-in-love-with-julia/main?filepath=08%20graphs%20-%2001%20introduction.ipynb)

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.1-darkmode.webp">
</a>

# Fall-in-love-with-Julia: Graphs in Julia with Graphs.jl

a 101 introduction session

In [ ]:
using Random; Random.seed!(2022);  # make sure this tutorial is reproducible

# Graphs.jl - Introduction

"The project goal is to mirror the functionality of robust network and graph analysis libraries such as NetworkX." (https://github.com/JuliaGraphs/Graphs.jl)

In this introductory notebook we are going to look into how to construct and inspect Graphs.

<img src="https://juliagraphs.org/Graphs.jl/dev/assets/logo.png" width="30%">

### What Graphs.jl does:
- defines Graph structure
- undirected graphs `Graph` and directeted graphs `DiGraph` 

### What Graphs.jl does not:
- stores attributes and other information - these should be stored outside of the graph itself

In [ ]:
using Graphs, GraphPlot

# add_vertices! & add_edge!

In [ ]:
G1 = Graph(1, 0) # graph with 1 vertices and 0 edges

In [ ]:
add_vertices!(G1, 2)  # be careful, add_vertices is not idempotent!
G1 # graph with 3 vertices and 0 edges

In [ ]:
# Make a line
add_edge!(G1, 1, 2)
add_edge!(G1, 1, 3)
G1 # graph with 3 vertices and 2 edges

In [ ]:
gplot(G1, nodelabel=1:3)

In [ ]:
G1[[Edge(1,3)]]

### Graph implementation details

don't use this, but still enlightening

In [ ]:
fieldnames(typeof(G1))

In [ ]:
G1.ne

In [ ]:
G1.fadjlist

### Graph properties

In [ ]:
neighbors(G1, 1)

In [ ]:
common_neighbors(G1, 2, 3)

In [ ]:
degree(G1, 1)

In [ ]:
@show ne(G1)
@show nv(G1)

gplot(G1, nodelabel=1:nv(G1), edgelabel=1:ne(G1))

In [ ]:
@show [vertices(G1)...];
@show [edges(G1)...];

In [ ]:
incidence_matrix(G1)

In [ ]:
adjacency_matrix(G1)

In [ ]:
laplacian_matrix(G1)

documentation of more properties see
- https://juliagraphs.org/Graphs.jl/dev/basicproperties/ 
- https://docs.juliahub.com/Graphs/VJ6vx/1.4.0/core/

## 💪 it is your turn: Make a triangle

transform G1 so that it becomes a triangle

In [ ]:
# ...

# Adjacency matrix & rem_vertex!

In [ ]:
A = [
    0 1 1
    1 0 1
    1 1 0
]
G2 = Graph(A)
gplot(G2, nodelabel=1:nv(G2), edgelabel=1:ne(G2))

In [ ]:
G1 == G2

In [ ]:
rem_vertex!(G2, 1)
gplot(G2, nodelabel=1:nv(G2), edgelabel=1:ne(G2))

Caution: If removing vertices, the vertex numbers are re-ordered!

If you want to link data to the graph, make sure you use `MetaGraph` from [MetaGraphs.jl](https://github.com/JuliaGraphs/MetaGraphs.jl)

# Create random graphs

In [ ]:
gplot(Graph(4, 3), nodelabel=1:4)

In [ ]:
gplot(DiGraph(4, 6), nodelabel=1:4)

In [ ]:
gplot(smallgraph("house"))

## 💪 it is your turn: Plot another smallgraph

you may want to take a look at the documentation of `smallgraph` by typing `?smallgraph`

In [ ]:
# ...

some other random plots

In [ ]:
gplot(complete_digraph(5))

In [ ]:
gplot(clique_graph(3, 4))

There are many many more **generators** for graphs. All documented at https://juliagraphs.org/Graphs.jl/dev/generators/#Graph-Generators

# Operators - a set-like interface

The current documentation of Operators seems corrupt, better take a look at a previous version: https://docs.juliahub.com/Graphs/VJ6vx/1.4.0/operators/

In [ ]:
regular = random_regular_graph(3, 2)
display(gplot(regular))

tree = binary_tree(3)
display(gplot(tree))

gplot(cartesian_product(regular, tree))

In [ ]:
gplot(blockdiag(path_graph(3), path_graph(2)))

## 💪 it is your turn: Combine blockdiag (can be seen as sum) and cartesian_product

and see how the graph adapts respectively

In [ ]:
# ...

### some other graph operators

In [ ]:
gplot(tensor_product(path_graph(3), random_regular_graph(3, 2)))

In [ ]:
gplot(complement(path_graph(5)))

And many more, like `egonet`, `induced_subgraph`, ... See https://docs.juliahub.com/Graphs/VJ6vx/1.4.0/operators/

# Reading/writing graphs

In [ ]:
saveme = erdos_renyi(5, 0.3)
gplot(saveme)

In [ ]:
savegraph("mygraph.lgz", saveme)  # z = compressed, lg=LightGraph

In [ ]:
gplot(loadgraph("mygraph.lgz"))

In [ ]:
;cat "mygraph.lgz"

### Other graph formats

[GraphIO.jl](https://github.com/JuliaGraphs/GraphIO.jl) supports reading EdgeList, GML, Graph6, GraphML, Pajek NET, DOT and CDF files.

In [ ]:
gml = """
graph [
    comment "This is a sample graph"
    directed 1
    id 42
    node [
        id 1
        label "node 1"
        thisIsASampleAttribute 42
    ]
    node [
        id 2
        label "node 2"
        thisIsASampleAttribute 43
    ]
    node [
        id 3
        label "node 3"
        thisIsASampleAttribute 44
    ]
    edge [
        source 1
        target 2
        label "Edge from node 1 to node 2"
    ]
    edge [
        source 2
        target 3
        label "Edge from node 2 to node 3"
    ]
    edge [
        source 3
        target 1
        label "Edge from node 3 to node 1"
    ]
]
"""

open("mygraph.gml", "w") do io
   write(io, gml)
end;

In [ ]:
import GraphIO  # Graph6 NET Edgelist and CDF work out of the box
import ParserCombinator  # needed in addition for DOT or GML
# import CodecZlib  # needed for LGCompressed
# import EzXML  # needed for GEXF and GraphML

gplot(loadgraph("mygraph.gml", "digraph", GraphIO.GMLFormat()))
# IMPORTANT! "digraph" because of directed graph

In [ ]:
dot = """
digraph mygraph {
    a -> b -> c;
    b -> d;
}
"""

open("mygraph.dot", "w") do io
   write(io, dot)
end;

In [ ]:
# it is important that the graph is named
gplot(loadgraph("mygraph.dot", "mygraph", GraphIO.DOTFormat()))

Many graph file formats have some special pecularities. In case something does not work out of the box, the best documentation available for GraphIO are the test runs.
Find them at https://github.com/JuliaGraphs/GraphIO.jl/tree/master/test

----

# Full example - global cascades on random networks

Simple simulation how easy it for a post to go viral in a network. Known as Watts-model.
Adapted from here https://nbviewer.org/github/JuliaGraphs/JuliaGraphsTutorials/blob/master/Watts-Model.ipynb

In [ ]:
"""
Computes the fraction of neighbors engaged within the neighborhood
of a given node.
"""
function fraction_engaged(node::Int,
                          G::Graph,
                          engagement::BitVector)
    num_engaged_neighbors = 0
    for nbr in neighbors(G, node)
        if engagement[nbr] == true
            num_engaged_neighbors += 1
        end
    end
    return num_engaged_neighbors / length(neighbors(G, node))
end

In [ ]:
"""
Updates the node_status of all vertices, one iteration only.
"""
function update_engagement!(G::Graph,
                            engagement::BitVector,
                            threshold::Float64)
    for node in Random.shuffle(vertices(G))
        if engagement[node] == false
            if fraction_engaged(node, G, engagement) > threshold
                engagement[node] = true
            end
        end
    end

    return nothing
end

In [ ]:
import StatsBase
"""
Executes the simulation

Output
-----------
A vector of number of engaged nodes at the end of each realization
of the simulation

Hyper Parameters of the model
----------
1. Number of nodes in the Watts-Strogatz graph (n)
2. Average degree (z)
3. Threshold (a specific value)
4. Time steps for simulation to be run (T)
5. Number of realizations
"""
function simulation(; n::Int, z::Int, threshold::Float64, T::Int, n_realizations::Int)
    output = Vector{Int}(undef, n_realizations)
    beta = z/n

    for r in 1:n_realizations
        G = watts_strogatz(n, z, beta)
        # Select a single random node from the network and seed it
        engagement = falses(nv(G))
        engagement[StatsBase.sample(vertices(G))] = true

        # Update the network for predefined number of time steps
        for _ in 1:T
            update_engagement!(G, engagement, threshold)
        end
        output[r] = sum(engagement)
    end

    return output
end

In [ ]:
n = 10^4
z = 5
threshold = 0.18
T = 50
n_realizations = 100

In [ ]:
beta = z/n
gplot(watts_strogatz(n, z, beta))

In [ ]:
data1 = simulation(; n, z, threshold, T, n_realizations)
histogram(data1, xlab="Number of engaged nodes", ylab="Frequency", legend=false)

In [ ]:
z2 = 6;  # +1

In [ ]:
data2 = simulation(; n, z=z2, threshold, T, n_realizations);
histogram(data2, xlab="Number of engaged nodes", ylab="Frequency", legend=false)

As can be seen from these two graphs, no global cascade occurs in the second case, while there are a few in the first! It is remarkable that just increasing the average degree of the network by 1 changes the entire outcome of the diffusion process.

The code presented here can be used to reproduce all the results discussed in Watts (2002).

## 💪 it is your turn: Play with the above hyperparameters

and see how the results change

----

# Take a look at part II - Graphs.jl Computation

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/schlichtanders/fall-in-love-with-julia/main?filepath=08%20graphs%20-%2002%20computation.ipynb)

# Thank you for joining

for questions or suggestions please contact me at stephan.sahm@jolin.io

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.1-darkmode.webp">
</a>